Bibliotecas


In [160]:
import pandas as pd
import time
import numpy as np
from sklearn.model_selection import KFold
import math
from sklearn.metrics import mean_absolute_error,mean_squared_error,recall_score,precision_score,f1_score
from sklearn.metrics.pairwise import cosine_similarity
pd.options.mode.chained_assignment = None 


Lendo dataframes


In [161]:
dataOriginal=pd.read_csv('./ml-100k/u.data',sep=r'\t',names=['user id','item id', 'rating', 'timestamp'])
filmesId=pd.DataFrame((set(dataOriginal['item id'])),columns=['Filmes'])

itemData=pd.read_csv('./ml-100k/u.item',sep=r'|',names=['movie_id ', 'movie title' , 'release date' ,' video release date' ,
              'IMDb URL' ,' unknown' , 'Action' , 'Adventure' , 'Animation' ,
             ' Childrens ', 'Comedy' , 'Crime' , 'Documentary' , 'Drama' , 'Fantasy' ,
              'Film-Noir' , 'Horror' , 'Musical' , 'Mystery' , 'Romance' , 'Sci-Fi' ,
              'Thriller' , 'War' , 'Western' ], encoding='latin-1')
userData=pd.read_csv('./ml-100k/u.user',sep='|',names=['user id','age','Genero','Ocupacao','ZipCode'])

C:\Users\User\AppData\Local\Temp\ipykernel_25908\1420820392.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  dataOriginal=pd.read_csv('./ml-100k/u.data',sep=r'\t',names=['user id','item id', 'rating', 'timestamp'])


In [162]:
userItem=dataOriginal.pivot(index='user id',columns='item id',values='rating').fillna(0)
userItem[:]=0
display(userItem)
userItem=userItem.to_numpy()
userItem

item id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
940,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [163]:
#iniciando
similaridade = pd.DataFrame(index=userData['user id'],columns=userData['user id'])
for id in userData['user id']:
    similaridade[id][id]=1
display(similaridade)
similaridade=similaridade.to_numpy()

user id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
user id,,,,,,,,,,,,,,,,,,,,,
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
941,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN


In [164]:
def h(rat,user,mediaUser):
    val=rat - mediaUser[user-1]
    return val

In [165]:
def  cossine_Similarity(u, v,tabelaSimilaridade,id_u,id_v):
    itemsComuns = np.where(((u!=0) & (v!=0)))[0]
    j=u[itemsComuns]
    k=v[itemsComuns]
    x=u[u!=0]
    y=v[v!=0]
    if(u.size==0 or v.size==0):
        return 0
    result = np.dot(j, k)/(np.linalg.norm(x)*np.linalg.norm(y))

    tabelaSimilaridade[id_u-1][id_v-1]=result
    tabelaSimilaridade[id_v-1][id_u-1]=result
    return result

In [166]:
def  pearson(u, v,tabelaSimilaridade,id_u,id_v):
    itemsComuns = np.where(((u!=0) & (v!=0)))[0]
    j=u[itemsComuns]
    k=v[itemsComuns]
    x=u[u!=0]
    y=v[v!=0]
    mediaU=np.mean(u)
    mediaV=np.mean(v)
    t=j-mediaU
    z=k-mediaV
    l=x-mediaU
    m=y-mediaV
    if(u.size==0 or v.size==0):
        return 0
    result = np.dot(t, z)/(np.linalg.norm(l)*np.linalg.norm(m))

    tabelaSimilaridade[id_u-1][id_v-1]=result
    tabelaSimilaridade[id_v-1][id_u-1]=result
    return result

In [167]:
def user_based(user_id, item_id,data,tabelaSimilaridade,mediaUser,k,similarityFunction,popularidade):
        values_to_match = np.unique(data[:, item_id-1 ])
        values_to_match = values_to_match[values_to_match != 0]
        atching_indices = np.where(np.isin(data[:, item_id-1], values_to_match))[0]
        atching_indices+=1
        Nui = pd.DataFrame(userData[userData['user id'].isin(atching_indices)]['user id'].values,columns=['user id']).reset_index(drop=True)
        if(len(Nui)<=popularidade):
            return mediaUser[user_id-1]
        if similarityFunction=='cosine':
            # display(Nui)
            Nui['similaridade']=Nui['user id'].apply(lambda x:(
                                cossine_Similarity(data[user_id-1][:],
                                                    data[x-1][:],tabelaSimilaridade,user_id,x
                                                    )
                                                    if math.isnan(tabelaSimilaridade[user_id-1][x-1])else tabelaSimilaridade[user_id-1][x-1]
                                                    ))
    
        elif similarityFunction=='pearson':
            Nui['similaridade']=Nui['user id'].apply(lambda x:(
                                pearson(data[user_id-1][:],
                                        data[x-1][:],tabelaSimilaridade,user_id,x
                                        )
                                        if math.isnan(tabelaSimilaridade[user_id-1][x-1])else tabelaSimilaridade[user_id-1][x-1]))
        
        Nui=Nui[Nui['similaridade']!=0]
        if(len(Nui)<=popularidade):
            return mediaUser[user_id-1]
        Nui=Nui.sort_values(by='similaridade',ascending=False).head(k)
    
        parcial1=sum(Nui.apply(lambda x:
                                np.dot(x['similaridade'],h(data[int(x['user id'])-1][item_id-1],int(x['user id']),mediaUser)),axis=1
                                ))
        parcial2 = sum(Nui.apply(lambda x:
                                (abs(x['similaridade']) ),axis=1
                                ))
        val=mediaUser[user_id-1]
        resp=(parcial1/parcial2)+val
        # print(resp)    
        return resp

In [168]:
class MemoryBased():

    def __init__(self, tabelaSimilaridade,userItem,k=5,popularityInity=5,metrica='user_based',calculaSimilaridade='cosine'):
        self.tabelaSimilaridade = tabelaSimilaridade
        self.userItem=userItem
        self.k=k
        self.metrica=metrica
        self.calculaSimilaridade=calculaSimilaridade
        self.PopularityInity=popularityInity

    def fit(self, x_train, y_train):
        x_train['rating']=y_train
        for index, row in x_train.iterrows():
            user_id=row['user id']
            item_id=row['item id']
            self.userItem[user_id-1][item_id-1]=row['rating']
        # display(self.userItem)
    def predict(self, x):
        y_pred = []
        mediaUser=self.userItem.mean(axis=1)
        for index, row in x.iterrows():
            user_id=row['user id']
            item_id=row['item id']
            if(self.metrica=='user_based'):
                # display(self.calculaSimilaridade)
                if self.calculaSimilaridade=='cosine':
                    
                    temp=user_based(user_id,item_id,self.userItem,self.tabelaSimilaridade,mediaUser,self.k,self.calculaSimilaridade,self.PopularityInity)
                    y_pred.append(temp)
                elif self.calculaSimilaridade=='pearson':
                    temp=user_based(user_id,item_id,self.userItem,self.tabelaSimilaridade,mediaUser,self.k,self.calculaSimilaridade,self.PopularityInity)
                    y_pred.append(temp)
                else:
                    display('Métrica de similaridade não encontrada')
        return y_pred
        

In [169]:
class ModelBased():
    def __init__(self, userItem, num_factors=100, learning_rate=0.005, reg_param=0.02, num_epochs=20, metric='regularized SVD') :
        self.num_factors = num_factors
        self.learning_rate = learning_rate
        self.reg_param = reg_param
        self.num_epochs = num_epochs
        self.metric = metric
        self.userItem = userItem
    def fit(self,x_train,y_train):
        x_train['rating']=y_train
        for index, row in x_train.iterrows():
            user_id=row['user id']
            item_id=row['item id']
            self.userItem[user_id-1][item_id-1]=row['rating']
        num_users, num_items = self.userItem.shape
        self.user_factors = np.random.normal(scale=1.0/self.num_factors, size=(num_users, self.num_factors))
        self.item_factors = np.random.normal(scale=1.0/self.num_factors, size=(num_items, self.num_factors))
        self.bu=np.zeros(num_users)
        self.bi=np.zeros(num_items)
        self.MediaGlobal=self.userItem.mean()
        for epoch in range(self.num_epochs):
            for i in range(num_users):
                for j in range(num_items):
                    if self.userItem[i, j] > 0:  # only consider non-zero ratings
                        error = self.userItem[i, j] - self.predictIntern(i, j)
                        self.bu[i] += self.learning_rate * (error - self.reg_param * self.bu[i])
                        self.bi[j] += self.learning_rate * (error - self.reg_param * self.bi[j])
                        self.user_factors[i, :] += self.learning_rate * (error * self.item_factors[j, :] - self.reg_param * self.user_factors[i, :])
                        self.item_factors[j, :] += self.learning_rate * (error * self.user_factors[i, :] - self.reg_param * self.item_factors[j, :])
    
    def predictIntern(self, user, item):
        return self.MediaGlobal+self.bi[item]+self.bu[user]+np.dot(self.user_factors[user, :], self.item_factors[item, :])
    def predict(self, X_test):
        y_pred = []
        for index, row in X_test.iterrows():
            user_id=row['user id']
            item_id=row['item id']
            y_pred.append(self.predictIntern(user_id-1,item_id-1))
        return y_pred

        

In [170]:

total=dataOriginal.drop(columns='timestamp')
total=total.sort_values(by='user id')
x=total.drop(columns='rating')
y=total['rating']
kf=KFold(n_splits=5, shuffle=True, random_state=42)

In [171]:
timeMemory=[]
timeModel=[]
maeMemory=[]
rmseMemory=[]
maeModel=[]
rmseModel=[]
ks=[10,20,30,40,50]
popularity=[0,1,2,3,]
calculoSimilaridade=['cosine','pearson']
num_factors=[10,20,30,40,50,60,70,80,90,100]
learning_rate=[0.001,0.002,0.003,0.004,0.005,0.006,0.007,0.008,0.009,0.01]
reg_param=[0.001,0.002,0.003,0.004,0.005,0.006,0.007,0.008,0.009,0.01]
num_epochs=[10,20,30,40,50]
bestk=10
bestpopularity=0
bestcalculoSimilaridade='cosine'
bestnum_factors=100
bestlearning_rate=0.001
bestreg_param=0.001
bestnum_epochs=10



In [172]:
infosK=pd.DataFrame(columns=['K','MAE','RMSE','Time']).set_index('K')
for k in ks:
    rmseMemory=[]
    maeMemory=[]
    timeMemory=[]
    for train_index, test_index in kf.split(x):
        similaridadeCopy=similaridade.copy()
        userItemCopy=userItem.copy()
        startTimeTotal=time.time()
        userBased=MemoryBased(similaridadeCopy,userItemCopy,k=k,popularityInity=bestpopularity,metrica='user_based',calculaSimilaridade=bestcalculoSimilaridade)
        x_train, x_test = x.iloc[train_index], x.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        userBased.fit(x_train,y_train)
        prdict=userBased.predict(x_test)
        endTimeTotal=time.time()
        maeMemory.append(mean_absolute_error(y_test,prdict))
        rmseMemory.append(math.sqrt(mean_squared_error(y_test,prdict)))
        timeMemory.append(endTimeTotal-startTimeTotal)
    infosK.loc[k]=[np.mean(maeMemory),np.mean(rmseMemory),np.mean(timeMemory)]

infosK['rmse_norm'] = (infosK['RMSE'] - infosK['RMSE'].min()) / (infosK['RMSE'].max() - infosK['RMSE'].min())
infosK['mae_norm'] = (infosK['MAE'] - infosK['MAE'].min()) / (infosK['MAE'].max() - infosK['MAE'].min())

# Criar uma métrica combinada
infosK['combined'] = infosK['rmse_norm'] + infosK['mae_norm']
bestk=infosK['combined'].idxmin()
infosK=infosK.drop(columns=['rmse_norm','mae_norm','combined'])
display(infosK)

,MAE,RMSE,Time,rmse_norm,mae_norm,combined
K,,,,,,
10,0.831411,1.038713,36.048033,1.000000,1.000000,2.000000
20,0.821252,1.025580,37.279316,0.102741,0.117122,0.219863
30,0.819905,1.024076,39.520584,0.000000,0.000000,0.000000
40,0.820262,1.024900,41.113982,0.056250,0.031077,0.087327
50,0.821075,1.026049,43.980066,0.134766,0.101703,0.236469


In [ ]:
Kmin=pd.DataFrame(columns=['K','MAE','RMSE','Time']).set_index('K')
for k in popularity:
    rmseMemory=[]
    maeMemory=[]
    timeMemory=[]
    for train_index, test_index in kf.split(x):
        similaridadeCopy=similaridade.copy()
        userItemCopy=userItem.copy()
        startTimeTotal=time.time()
        userBased=MemoryBased(similaridadeCopy,userItemCopy,k=bestk,popularityInity=k,metrica='user_based',calculaSimilaridade=bestcalculoSimilaridade)
        x_train, x_test = x.iloc[train_index], x.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        userBased.fit(x_train,y_train)
        prdict=userBased.predict(x_test)
        endTimeTotal=time.time()
        maeMemory.append(mean_absolute_error(y_test,prdict))
        rmseMemory.append(math.sqrt(mean_squared_error(y_test,prdict)))
        timeMemory.append(endTimeTotal-startTimeTotal)
    Kmin.loc[k]=[np.mean(maeMemory),np.mean(rmseMemory),np.mean(timeMemory)]

Kmin['rmse_norm'] = (Kmin['RMSE'] - Kmin['RMSE'].min()) / (Kmin['RMSE'].max() - Kmin['RMSE'].min())
Kmin['mae_norm'] = (Kmin['MAE'] - Kmin['MAE'].min()) / (Kmin['MAE'].max() - Kmin['MAE'].min())

# Criar uma métrica combinada
Kmin['combined'] = Kmin['rmse_norm'] + Kmin['mae_norm']
bestk=Kmin['combined'].idxmin()
display(Kmin)

In [ ]:
# for train_index, test_index in kf.split(x):
#     similaridadeCopy=similaridade.copy()
#     userItemCopy=userItem.copy()
#     startTimeTotal=time.time()
#     userBased=MemoryBased(similaridadeCopy,userItemCopy,k=40,popularityInity=1,metrica='user_based',calculaSimilaridade='cosine')
#     x_train, x_test = x.iloc[train_index], x.iloc[test_index]
#     y_train, y_test = y.iloc[train_index], y.iloc[test_index]
#     userBased.fit(x_train,y_train)
#     prdict=userBased.predict(x_test)
#     endTimeTotal=time.time()
#     maeMemory.append(mean_absolute_error(y_test,prdict))
#     rmseMemory.append(math.sqrt(mean_squared_error(y_test,prdict)))
#     display(maeMemory)
#     display(rmseMemory)
#     timeMemory.append(endTimeTotal-startTimeTotal)
# mediaMaeMemory=sum(maeMemory)/len(maeMemory)
# mediaRmseMemory=sum(rmseMemory)/len(rmseMemory)
# mediaTimeMemory=sum(timeMemory)/len(timeMemory)

In [ ]:
# display('MemoryBased')
# display(maeMemory)
# display(rmseMemory)
# display(timeMemory)
# display('MemoryBased')
# display('MAE: '+str(mediaMaeMemory))
# display('RMSE: '+str(mediaRmseMemory))
# display('Time: '+str(mediaTimeMemory))

In [ ]:
# for train_index, test_index in kf.split(x):
#     userItemCopy=userItem.copy()
#     startTimeTotal=time.time()
#     model=ModelBased(userItemCopy,num_factors=10, learning_rate=0.01, reg_param=0.02, num_epochs=20, metric='regularized SVD')
#     x_train, x_test = x.iloc[train_index], x.iloc[test_index]
#     y_train, y_test = y.iloc[train_index], y.iloc[test_index]
#     model.fit(x_train,y_train)
#     prdict=model.predict(x_test)
#     endTimeTotal=time.time()
#     maeModel.append(mean_absolute_error(y_test,prdict))
#     rmseModel.append(math.sqrt(mean_squared_error(y_test,prdict))
#     )
#     timeModel.append(endTimeTotal-startTimeTotal)
# mediaMaeModel=sum(maeModel)/len(maeModel)
# mediaRmseModel=sum(rmseModel)/len(rmseModel)
# mediaTimeModel=sum(timeModel)/len(timeModel)


In [ ]:

# display('ModelBased')
# display('MAE: '+str(mediaMaeModel))
# display('RMSE: '+str(mediaRmseModel))
# display('Time: '+str(mediaTimeModel))

In [ ]:

# display('ModelBased')
# display(maeModel)
# display(rmseModel)
# display(timeModel)
